In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
final_fig_path = "../Figures/"
from time import time

In [2]:
# %%writefile fun_response_funs.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp

    
def fun_response(x,M1,M2,index,a1,a2,h1,h2,**params):
    '''
    functional response to prey as a function of predator group size (x) and 
    prey population sizes (M1, M2)
    a_iα_i(x)M_i/(1 + h_1 a_1 α_1(x) M_1 + h_2 a_2 α_2(x) M_2)
    
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    index - 1 (big prey) or 2 (small prey)
    a1 - attack rate of big prey
    a2 - attack rate of small prey
    h1 - handling time per big prey caught
    h2 - handling time per small prey caught
    params: a dictionary of other parameters, that at least must include α1_of_1, α2_of_1, s1, s2

    @returns
    functional response for prey type <index> (a float)

    @examples
    >>fun_response(x=1,M1=10,M2=10,index=1,a1=1,a2=1,h1=0.5,h2=0.5, 
                    **dict(α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, s2 = 2) )
    0.08333333333333336
    (answer should be 10*a1*α1_of_1/(1+h1*a1*α1_of_1*10 + h2*a2*α2_of_1*10) = 0.08333333333333333
    
    '''
    
    α1 = fun_attack_rate(x,1,**params)
    α2 = fun_attack_rate(x,2,**params)
    if index == 1:
        numerator = a1*α1*M1
    elif index == 2:
        numerator = a2*α2*M2
    denominator = 1 + a1*α1*h1*M1 + a2*α2*h2*M2
    return numerator/denominator


def fun_attack_rate(x, index, α1_of_1, α2_of_1, s1, s2, **params):
    '''
    The attack rate as a function of x
    
    @inputs:
    x: group size, 1,2,3,...
    index: 1 or 2, indicates prey type 1 (big prey) or 2 (small prey)
    α1_of_1: the attack rate of big prey for group size 1
    α2_of_1: the attack rate of small prey for group size 1
    s1: critical group size for big prey, must be >= 2
    s2: critical group size for small prey, must be >= 2
    
    @returns:
    attackrate (a float)

    @example:
    >> fun_attack_rate(1,2,0.05,0.95,2,2,**dict())
    0.9500000000000001
    >> fun_attack_rate(1,1,0.05,0.95,2,2,**dict())
    0.05000000000000001
    
    '''
    if index == 1:
        θ_1 = - np.log(1/α1_of_1 - 1)/(1-s1)
        return 1/(1 + np.exp(- θ_1 * (x - s1)))
    elif index == 2:
        θ_2 = - np.log(1/α2_of_1 - 1)/(1-s2)
        return 1/(1 + np.exp(- θ_2 * (x - s2)))
    

In [17]:
%%writefile fitness_funs.py

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *


def fun_fitness(x, M1, M2, **params):
    '''
    this is a subordinate's fitness of being in a group of size x
    @inputs:
    x - grp size
    M1 - pop size of big prey
    M2 - pop size of small prey
    params - a dictionary at least including: b1, b2, r, γ, a1, a2, h1, h2, α1_of_1, 
                                                    α2_of_1, s1, s2, limited_portions
                                                    (and must also have b0 if limited_portions = True)
    @returns:
    fitness of a subordinate, a float (or array if one of the parameter inputs is an array)

    @example:
    >>fun_fitness(x=np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, 
                                                    limited_portions = False))
    array([0.24166667, 0.45833333, 0.53055556])
    '''
    fitnesses = fitness_from_prey(x, M1, M2,**params)
    total_sub_fitness = np.sum(fitnesses,0)
    return total_sub_fitness



def per_capita_fitness_from_prey(x,M1,M2,b1,b2,limited_portions, **params):
    '''
    portion of direct fitness from each prey type without any skew, stored in an array
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    limited_portions - True or False, whether predators can only eat a limited amount or not
    params - dictionary of other parameters, which must at least contain 
             a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2

    @returns:
    np.array([<inclusive fitness from big prey>, <inclusive fitness from small prey>])
    (so the rows correspond to prey types

    @example
    >>per_capita_fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, 
                                                    limited_portions = False))
    array([[0.08333333, 0.41666667, 0.52777778],
       [0.15833333, 0.04166667, 0.00277778]])
    
    >>per_capita_fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2,b0 = 0.05, 
                                                    limited_portions = True))
    array([[0.00416667, 0.04166667, 0.07916667],
       [0.07916667, 0.04166667, 0.00277778]])
    '''
    # set portion size, need to account for x being an array
    conversion_big, conversion_small = conversion_prey(x,b1,b2,limited_portions,**params)

        
    w_per_capita = np.array([conversion_big*fun_response(x,M1,M2,1,**params), 
                         conversion_small*fun_response(x,M1,M2,2,**params)])
    return w_per_capita
    
def fitness_from_prey(x, M1, M2,b1, b2, r, γ,limited_portions,**params):
    '''
    portion of inclusive fitness from each prey type, stored in an array, after potentially unequal sharing
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    r - relatedness between group members
    γ - extent of reproductive skew (portion of subordinate's food donated to dominant)
    params - dictionary of other parameters, which must at least contain 
             a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2, b1, b2, limited_portions

    @returns:
    np.array([<inclusive fitness from big prey>, <inclusive fitness from small prey>])
    (so the rows correspond to prey types

    @example
    >>fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, 
                                                    limited_portions = False))
    array([[0.08333333, 0.41666667, 0.52777778],
       [0.15833333, 0.04166667, 0.00277778]])
    
    >>fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2,b0 = 0.05, 
                                                    limited_portions = True))
    array([[0.00416667, 0.04166667, 0.07916667],
       [0.07916667, 0.04166667, 0.00277778]])
    '''
    # set portion size, need to account for x being an array

        
    w_per_capita = per_capita_fitness_from_prey(x,M1,M2, **params)
    try:
        if x > 1:
            repro_exchange = (1-γ)*(1-r) + r*x
            return w_per_capita * repro_exchange
        else:
            return w_per_capita
    except ValueError:
        repro_exchange = np.ones(np.shape(x))
        repro_exchange[x>1] = (1-γ)*(1-r) + r*x[x>1]
        return  w_per_capita * repro_exchange

def conversion_prey(x,b1,b2,limited_portions,**params):
    '''
    Finds the number of predators produced per prey item caught, for big prey and small prey
    @inputs:
    x - pred group size, could be vector
    b1 - number preds produced from big prey if completely eaten
    b2 - number preds produced from small prey if completely eaten
    limited_portions- whether there's a limit of how much food one predator can eat
    params - dic of parameters used by rest of model
    
    @returns
    conversion_big, conversion_small (both floats)

    @examples
    >> conversion_prey(x=2,b1=1,b2=0.1,limited_portions=True,**dict(b0 = 0.1))
    (0.1, 0.05)
    >> conversion_prey(x=[1,2],b1=1,b2=0.1,limited_portions=True,**dict(b0 = 0.1))
    (array([0.1, 0.1]), array([0.1 , 0.05]))
    >> conversion_prey(x=2,b1=1,b2=0.1,limited_portions=False,**dict(b0 = 0.1))
    (0.5, 0.05)
    '''
    if limited_portions == True:
        b0 = params["b0"]
        try:
            conversion_big = b0 if b1/x > b0 else b1/x
            conversion_small = b0 if b2/x > b0 else b2/x 
        except (ValueError,TypeError): # if x is a list (-->TypeError) or an ndarray (-->ValueError)
            x = np.array(x)
            conversion_big = np.ones(np.shape(x))
            conversion_small = conversion_big.copy()
            conversion_big = b1/x
            conversion_big[conversion_big>b0] = b0
            conversion_small = b2/x
            conversion_small[conversion_small>b0] = b0
    else:
        conversion_big = b1/x
        conversion_small = b2/x
    return conversion_big, conversion_small
    
        
        
def fun_fitness_from_big_prey(x, M1, M2, **params):
    '''
    portion of inclusive fitness from big prey type. calls fitness_from_prey
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    params - dictionary of other parameters, which must at least contain 
             r, γ, a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2, limited_portions
             (and must also have b0 if limited_portions = True)

    @returns:
    <inclusive fitness from big prey>
    '''
    return fitness_from_prey(x, M1, M2, **params)[0]
def fun_fitness_from_small_prey(x, M1, M2, **params):
    '''
    portion of inclusive fitness from small prey type. calls fitness_from_prey
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    params - dictionary of other parameters, which must at least contain 
             r, γ, a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2, limited_portions
             (and must also have b0 if limited_portions = True)

    @returns:
    <inclusive fitness from small prey>

    @example
    >> fun_fitness_from_small_prey(1, 10, 10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
    >>                                    h1=0.5, h2=0.5, α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2,
                                          limited_portions = False))
    0.15833333333333333
    '''
    return fitness_from_prey(x, M1, M2, **params)[1]

Overwriting fitness_funs.py


In [3]:
np.array([1,2])*np.array([1,2])

array([1, 4])

In [ ]:
#%%writefile group_formation_funs.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *
from fitness_funs import *


def group_formation_model(t, f_of_x_vec,p,M1,M2,params):
    '''
    the full system of balance equations for x = 2,3,...,x_max
    @inputs:
    t - time, necessary for running with solve_ivp
    f_of_x_vec - a vector of the number of groups of size 1, 2, 3, ..., x_max (maximum group size)
    p - population size of predators
    M1 - population size of big prey
    M2 - population size of small prey
    params - is a dictionary of the parameters that must contain: 
            b1, b2,r, γ, a1, a2, h1, h2, α1_of_15, α2_of_1, s1, s2, limited_portions, 
            τx, δ, d
            (and b0 if limited_portions = False)
    '''
    x_max = params["x_max"]
    dfdt = [fun_dfdt(f_of_x_vec, x, p, M1, M2, **params) for x in range(2,x_max+1)]
    #for x in range(2,x_max+1):
    #    dfdt[x-2] = fun_dfdt(f_of_x_vec, x, p, M1, M2, **params)
    return dfdt

def group_formation_model_alt(t, f_of_x_vec,p,M1,M2, τx, δ, x_max, params):
    '''
    THIS IS BEING CONSTRUCTED
    I THINK I SHOULD HAVE SOMETHING THE TRANSITION MATRICES FIRST AS MUCH AS POSSIBLE BECAUSE IN PART THEY STAY THE SAME
    the full system of balance equations for x = 2,3,...,x_max
    @inputs:
    t - time, necessary for running with solve_ivp
    f_of_x_vec - a vector of the number of groups of size 1, 2, 3, ..., x_max (maximum group size)
    p - population size of predators
    M1 - population size of big prey
    M2 - population size of small prey
    params - is a dictionary of the parameters that must contain: 
            b1, b2,r, γ, a1, a2, h1, h2, α1_of_15, α2_of_1, s1, s2, limited_portions, 
            τx, δ, d
            (and b0 if limited_portions = False)
    '''
    def f(y):
        return f_of_x_vec[y-1] if y <= x_max else 0
    def D_tot(y):
        '''
        probability there is at least one death in group of size y
        '''
        return fun_1_death(y, τx, δ, **params)
    def D(z1,z2):
        # probability group of size z2 shrinks to group of size z1 for z1 \leq x_max  
        return fun_death_y_to_x(z1,z2, τx, δ, x_max, **params) if z1<z2 else 0
    def ϕ(y):
        # probability individual leaves group of size y for y <= x_max
        return fun_leave_group(y, M1, M2, x_max, **params) for y in range(2,x_max+1) else 0
    def ψ(y):
        return fun_join_group(y, M1, M2, x_max, **params) for y in range(1,x_max) else 0
    
    x_max = params["x_max"]
    dfdt = np.zeros(f_of_x_vec.shape)

    # births
    births = τx*sum([x*f(x)*per_capita_fitness_from_prey(x, M1, M2,**params) for x in range(1,x_max+1)])
    dfdt[0] = dfdt[0] + births

    #deaths - transition from one size
    death_transition_matrix = np.array([[D(x,y) for y in range(1,x_max+1)] for x in range(0,x_max)]) 
        # rows are size it shrinks to, columns are original size
    rate_size_x_after_deaths = np.matmul(death_transition_matrix,f_of_x_vec) # but 0th element 
                                                                            # is the number of size 0
    dfdt[1:] += rate_size_x_after_deaths[1:]
    
    # deaths - transition out of group of size x (could also do some along columns
    deaths_loss = np.array([D_tot(x) for x in range(1,x_max+1)])
    # TO DO: CHECK THIS IS THE SAME AS SUMMING ALONG COLUMNS OF death_transition_matrix
    rate_deaths_loss = f_of_x_vec*deaths_loss
    dfdt += rate_deaths_loss

    # groups shrink from individuals leaving
    
    
    return dfdt



def fun_dfdt(f_of_x_vec, x, p, M1, M2,  τx, δ, x_max, **params):
    '''
    fun_dfdt: This calculates the change in the distribution f(x) wrt time for x >= 2
        τx df/dt = -xf(x)ϕ(x) - f(1) f(x) ψ(x) - f(x) D(x) 
                + f(x+1)ϕ(x+1) + sum_{y=x+1}^{x_max} f(y) D(y)
        but slightly different for x = 2, x = x_max
    f(x) is the number of groups of size x
    @inputs
    f_of_x_vec - vector of f(x) for x = 1, 2, .., x_max.
    x - grp size, must be >= 2
    p - pred pop size
    M1 - big prey pop size
    M2 - small prey pop size
    τx - group size change time scale
    δ - pred death rate
    x_max - max grp size
    params - dictionary of params used by rest of model

    @returns
    float dfdt

    @example
    >> fun_dfdt(f_of_x_vec = [0,0], x=2, p=100, M1=0, M2=100, **dict(τx= 0.01, δ=0, x_max=3, 
    >>                                                          b1=1,b2=0.1,r=0, γ=0, a1=1, 
    >>                                                          a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, d = 100,
                                                        ))
    6.209899910586194e-26
    
    '''
    # get f(x), f(1), and f(x+1)
    
    def f(y):
        if y <= x_max:
            return f_of_x_vec[y-1]
        else:
            return 0
    def D_tot(y):
        '''
        probability there is at least one death in group of size y
        '''
        return fun_1_death(y, τx, δ, **params)
    def D(z1,z2):
        # probability group of size z2 shrinks to group of size z1 for z1 \leq x_max  
        return fun_death_y_to_x(z1,z2, τx, δ, x_max, **params)
    def ϕ(y):
        # probability individual leaves group of size y for y <= x_max
        return fun_leave_group(y, M1, M2, x_max, **params) 
    def ψ(y):
        return fun_join_group(y, M1, M2, x_max, **params)
    


    # if x = 2, τ_x df_dx, group formation is different. it is 1/2 f(1)^2 ψ(1) instead of f(x-1)f(1) ψ(x-1)
    # if x = x_max, the group cannot grow larger and there are no larger groups that can shrink to be that size
    
    if x == 1:
        births = τx*sum([x*f(x)*per_capita_fitness_from_prey(x, M1, M2,**params) for x in range(1,x_max+1)])
        individuals_leave_grps = sum([x*f(x)*ϕ(x) for x in range (2,x_max+1)])
        larger_grp_shrinks = 2*f(2)*ϕ(2)
        solitaried_die = f(1)*
    if x >= 2
        individual_leaves = x*f(x) * ϕ(x)
        grows_to_larger_group = f(1)*f(x) * ψ(x) if x<x_max else 0
        death_in_group = f(x) * D_tot(x)
        if x == 2:
            #join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
            join_smaller_grp = (1/2)*f(1)*(f(1)-1)*ψ(1)
        else:
            join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
        larger_grp_shrinks = (x+1)*f(x+1)*ϕ(x+1) if x < x_max else 0
        death_in_larger_grp = sum([f(y)*D(x,y) for y in range(x+1,x_max+1)]) if x < x_max else 0
        
    #dfdt_times_taux = x*f_of_x*fun_leave_group(x) - 
    
    
        return 1/τx * (-individual_leaves - grows_to_larger_group - death_in_group  
                  + join_smaller_grp + larger_grp_shrinks + death_in_larger_grp)

In [18]:
per_capita_fitness_from_prey?

Signature: per_capita_fitness_from_prey(x, M1, M2, b1, b2, limited_portions, **params)
Docstring:
portion of direct fitness from each prey type without any skew, stored in an array
@inputs:
x - pred group size
M1 - big prey pop size
M2 - small prey pop size
b1 - big prey  conversion (prey --> pred)
b2 - small prey conversion (prey --> pred)
limited_portions - True or False, whether predators can only eat a limited amount or not
params - dictionary of other parameters, which must at least contain 
         a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2

@returns:
np.array([<inclusive fitness from big prey>, <inclusive fitness from small prey>])
(so the rows correspond to prey types

@example
>>per_capita_fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                    h1=0.5, h2=0.5, 
                                                α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, 
                                       

In [14]:
fun_f_of_x(4, [1,2,1], x_max=3,**dict())

0

In [12]:
def fun_f_of_x(x, f_of_x_vec, x_max,**params):
    '''
    f(x)...find number of groups of size x
    @inputs
    x - grp size
    f_of_x_vec - vector of [f(1), f(2), f(3), ..., f(x_max)]
    p - pred pop size
    x_max - param, max grp size
    params - dictionary of params used by the rest of the model
    @ returns
    integer

    @examples
    >>fun_f_of_x(1, [1,2,1], x_max=4,**dict())
    1
    >>fun_f_of_x(4, [1,2,1], x_max=3,**dict())
    0
    
    '''
    if x <= x_max:
        return f_of_x_vec[x-1]
    else:
        return 0
    
    

def fun_leave_group(x, M1, M2, x_max, **params):
    '''
    The probability an individual leaves a group of size x.
    This is ϕ(x) in the text
    @inputs
    x - current grp size (before leaving)
    M1 - big prey pop size
    M2 - small prey pop size
    x_max - parameter, maximum group size
    params - dictionary of params used by the rest of the model
    '''
    # deciding between being alone and staying in group of size x
    return best_response_fun(1,x,M1,M2,**params)


def fun_join_group(x, M1, M2, x_max, **params):
    '''
    The probability an individual joins a group of size x.
    This is ψ(x) in the text
    @inputs
    x - current grp size (before joining)
    M1 - big prey pop size
    M2 - small prey pop size
    x_max - parameter, maximum group size
    params - dictionary of params used by the rest of the model
    '''
    # deciding between switching from being alone to being in a group of size x + 1
    return best_response_fun(x+1,1,M1,M2,**params)

def best_response_fun(x,y,M1,M2, d, **params):
    '''
    Compares W(x) to W(y) to "decide" on group size y or x
    @inputs
    x - potentially new group size
    y - current grp size
    M1 - big prey pop size
    M2 - small prey pop size
    d - steepness, or sensitivity, of best response function
    params - dictionary of params used by the rest of the model, 
            but must include all params relevant to functional responses and 
            inclusive fitness 
    @returns:
    float between 0 and 1

    @example
    >> best_response_fun(x=2,y=3,M1=10,M2=10, d=100, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2) )
    4.4162891945392386e-07
    >> 1 - best_response_fun(x=3,y=2,M1=10,M2=10, d=100, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2) )
    4.4162891943422267e-07
    
    '''
    W_of_x = fun_fitness(x,M1,M2, **params)
    W_of_y = fun_fitness(y, M1, M2, **params)
    return W_of_x**d/(W_of_x**d + W_of_y**d)

def nchoosek(n,k):
    '''
    n choose k
    n!/(k!(n-k)!)
    @inputs:
    n and k are integers, but can handle np.arrays
    @returns:
    positive integer (or array if inputs are arrays
    @example
    >> nchoosek(3,1)
    3.0
    >> nchoosek(np.array([3,2]),1)
    array([3.,2.])
    '''
    return sp.special.factorial(n)/(sp.special.factorial(k)*sp.special.factorial(n-k))


def fun_death_y_to_x(x, y, τx, δ, x_max, **params):
    '''
    The probability a group of size y shrinks to a group of size x because y - x individuals die, 
    works for for x < y, y <= x_max

    @inputs:
    x = group size after death
    y = original group size, y > x, y <= x_max
    δ = death rate
    x_max = maximum group size
    params = dictionary of other parameters used in the model

    @output:
    float between 0 and 1 (note for τx small, fun_death_y_to_x(x,y,**params) \approx 0 if x < y-1

    @example
    >>fun_death_y_to_x(x=2, y=3, **dict(τx=0.01, δ=0.1, x_max=10))
    0.0029940030000000003
    '''
    
    return nchoosek(y,y-x) * (δ*τx)**(y-x)*(1-δ*τx)**x

def fun_1_death(x, τx, δ, **params):
    '''
    The probability of AT LEAST one death in a group of size x over time τ_x
    @inputs:
    x - grp size
    τx - group evolution time constant
    δ - death rate
    params - dictionary of parameters from the ret of the model, not really needed...
    @output:
    float

    @example
    >> fun_1-death(x=1, **dict(τx = 0.01, δ = 0.1))
    0.0010000000000000009
    >> fun_1_death(3, 0.01, 0.1, **dict())
    0.002997000999999999
    
    '''
    return 1 - (1 - δ*τx)**x


    
    

In [ ]:
f_of_x_vec = [0,0]

In [60]:
fun_dfdt(f_of_x_vec = [0,0], x=2, p=100, M1=0, M2=100, **dict(τx= 0.01, δ=0, x_max=3, 
                                                                b1=1,b2=0.1,r=0, γ=0, a1=1, 
                                                                a2=1, h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, d = 100))

6.209899910586194e-26

In [34]:
fun_f_of_x(3,[0,1],3,x_max=3,**dict())

1

In [27]:
nchoosek(np.array([3,2]),1)

array([3., 2.])

In [ ]:
best_response_fun(2,3,M1,M2, d, **params)

In [12]:
fun_1_death(3,0.01, 0.1, **dict())

0.002997000999999999

In [5]:
fun_1_death(2,0.01, 0.1, **dict())

0.001998999999999973

In [6]:
1 - (1 - 0.01*0.1)**2

0.001998999999999973

In [ ]:
# %%writefile group_measures.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *
from fitness_funs import *
from group_formation_funs import *

def fun_num_groups(f_of_x_vec,p,x_max):
    total = 0
    for x in range(1,x_max+1):
        total += fun_f_of_x(x, f_of_x_vec, p, x_max,**dict())
    return total

'''
checks that dfdt = 0
needs to be checked
'''
def check_at_equilibrium(f_of_x_vec, p, M1, M2,  x_max, **params):
    for x in range(2, x_max+1):
        dfdt = fun_dfdt(f_of_x_vec, x, p, M1, M2, x_max=x_max,**params)
        at_equilibrium = np.abs(dfdt)<1e-5
        if not at_equilibrium:
            return 0
    return 1